<a href="https://colab.research.google.com/github/wisabd/CreditFraudML/blob/main/TrainingAndTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%store -r

In [20]:
!pip install sagemaker

from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3
import re
import sagemaker
import seaborn as sns

!pip install boto3
  # Installs boto3 as a *

from google.colab import userdata
import os
import warnings


# Set credentials as environment variables (they're loaded from Colab secrets)
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
aws_account_id = userdata.get('AWS_ACCOUNT_ID')  # Your stored account ID
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'  # Set your region

# Initialize boto3/SageMaker session and Define IAM role
import boto3
import re
import sagemaker
import seaborn as sns


# Suppress warnings (optional)
warnings.filterwarnings('ignore', category=UserWarning, module='sagemaker')

role_name = 'SageMaker-Execution-Role'  # Your role name

# Construct the full Role ARN
role_arn = f'arn:aws:iam::{aws_account_id}:role/{role_name}'
try:
    role = sagemaker.get_execution_role();  # Works in SageMaker notebooks
except ValueError:
    role = role_arn  # Fallback for Colab

#Manage interactions with the Amazon SageMaker APIs and any other AWS services needed.
#manipulating entities and resources that Amazon SageMaker uses, such as training jobs, endpoints, and input datasets in S3.

session = boto3.Session()  # Automatically picks up env variables
sm_session = sagemaker.Session(boto_session=session)
bucket = sm_session.default_bucket()

container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')
container

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3'

In [21]:
from sagemaker.inputs import TrainingInput
prefix = 'sagemaker/project-name'

# Replace s3_input with TrainingInput
s3_input_train = TrainingInput(
    s3_data=f's3://{bucket}/{prefix}/train',  # Using f-string (Python 3.6+)
    content_type='csv'
)

s3_input_validation = TrainingInput(
    s3_data=f's3://{bucket}/{prefix}/validation/',
    content_type='csv'
)

In [22]:
sm_session = sagemaker.Session(boto_session=session)

xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sm_session)
xgb.set_hyperparameters(
                        max_depth=5,
                        eta=0.1,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        alpha=0.5,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2025-04-23 10:19:38 Starting - Starting the training job...
2025-04-23 10:19:52 Starting - Preparing the instances for training...
2025-04-23 10:20:17 Downloading - Downloading input data...
2025-04-23 10:20:47 Downloading - Downloading the training image...
2025-04-23 10:21:28 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[10:21:44] 688x30 matrix with 20640 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Deter

In [23]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---------!

In [38]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

import pickle
import numpy as np
import boto3
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# Load test_data from Drive
with open('/content/drive/MyDrive/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

print(test_data)  # Verify

# Initialize the SageMaker predictor
try:
    # Replace with your actual endpoint name
    endpoint_name = 'sagemaker-xgboost-2025-04-23-10-22-29-065'

    # Create predictor with proper serialization
    xgb_predictor = Predictor(
        endpoint_name=endpoint_name,
        serializer=CSVSerializer(),
        content_type='text/csv'
    )

    # Verify endpoint exists
    sm_client = boto3.client('sagemaker')
    endpoint_status = sm_client.describe_endpoint(EndpointName=endpoint_name)['EndpointStatus']
    print(f"Endpoint status: {endpoint_status}")

except Exception as e:
    print(f"Error initializing predictor: {str(e)}")
    raise

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])

    for array in split_array:
        # Convert array to CSV string format expected by XGBoost
        csv_data = '\n'.join([','.join(map(str, row)) for row in array])

        try:
            # Get prediction and decode
            response = xgb_predictor.predict(csv_data)
            batch_preds = np.fromstring(response.decode('utf-8'), sep=',')
            predictions = np.concatenate([predictions, batch_preds])
        except Exception as e:
            print(f"Prediction error: {str(e)}")
            raise

    return predictions

# Make predictions (assuming test_data is a DataFrame with features)
try:
    predictions = predict(test_data.values[:, 1:])  # Skip first column if it's ID/target
    print(f"Successfully generated {len(predictions)} predictions")
    print(predictions[:10])  # Print first 10 predictions
except Exception as e:
    print(f"Prediction failed: {str(e)}")

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
        Class         V1        V2         V3        V4        V5        V6  \
16863       1  -2.787248 -0.071340  -1.505288  3.361777 -3.357422  0.565835   
41943       1  -2.140511  4.104871  -8.996859  4.028391 -5.131359 -4.153568   
1039        0  -1.684144  0.421962   2.601415 -0.412067  0.295015 -0.237649   
46909       1  -4.075975  0.963031  -5.076070  4.955963 -0.161437 -2.832663   
46773       0  -1.616933  0.332553   0.978147  1.272474  0.083678 -1.243975   
...       ...        ...       ...        ...       ...       ...       ...   
280149      1  -0.676143  1.126366  -2.213700  0.468308 -1.120541 -0.003346   
150697      1 -12.381048  8.213022 -16.962530  7.116091 -9.772826 -3.666836   
211505      0  -1.302118  0.859620   1.673845  2.456201  1.976591  0.186542   
171737      0   2.159317 -1.522302  -1.726698 -1.849936 -0.564546 -0.197293   
24

In [39]:
import pandas as pd
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

predictions,0.0,1.0
actual,,
0,55,1
1,3,40


In [40]:
%store prefix
%store bucket
%store s3_input_train
%store s3_input_validation

Stored 'prefix' (str)
Stored 'bucket' (str)
Stored 's3_input_train' (TrainingInput)
Stored 's3_input_validation' (TrainingInput)
